# Import Files & Libs

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount("/content/gdrive", force_remount=True)
!cp /content/gdrive/MyDrive/Decathlon/project.zip /content/
!unzip -q /content/project.zip
!rm /content/project.zip
!mkdir /content/results

In [59]:
import torch
import torch.nn
import numpy as np
import matplotlib.pyplot as plt

from model import lstm
from dataset import dataset
from training import training


import torch.optim as optim
import tqdm
import os
from importlib import reload
reload(lstm)
reload(dataset)
reload(training)

<module 'training.training' from 'c:\\Users\\Edouard\\github\\DecathlonTest\\training\\training.py'>

# Load Data

In [41]:
ds_train=dataset.Turnover_dataset(None)
ds_val=dataset.Turnover_dataset(None)
with open('/content/train_data.json', 'r') as f:
    ds_train.load_from_json(f)
with open('/content/val_data.json', 'r') as f:
    ds_val.load_from_json(f)

In [ ]:
ds_train.set_data_for_training(False)
ds_val.set_data_for_training(False)
ds_train.dep=73
ds_val.dep=73
dataloader_t = torch.utils.data.DataLoader(ds_train,batch_size=32,shuffle=True,drop_last=False) #Dataloader train set
dataloader_v = torch.utils.data.DataLoader(ds_val,batch_size=32,shuffle=True,drop_last=False) #Dataloader validation set

# Model

In [60]:
device= torch.device("cuda:0")
model = lstm.LSTM_Turnover(True,hidden_size=48,number_of_layer=4).to(device)

In [45]:
param=0
for key in model.state_dict():
    param+=model.state_dict()[key].view(-1).shape[0]
print(param)

30455


In [13]:
optimizer = optim.Adam(model.parameters(), lr=3e-4, betas=(0.9, 0.999)) #Adam Optimizer 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.3,verbose=True) #Learning rate scheduler
num_epochs=20

Adjusting learning rate of group 0 to 3.0000e-04.


In [ ]:
training.training(num_epochs,model,optimizer,scheduler,dataloader_t,dataloader_v,device,"/content/results/",warmup=0,future_pred=1)

In [ ]:
model.load_state_dict(torch.load("/content/results/checkpoint_6.pth")['model_state_dict'])
data=next(iter(dataloader_v))
future_pred=1
warmup=0
y=model( data["x"].float().to(device),
        data["annual_x"].float().to(device),
        data["annual_y"].float().to(device),
        future_pred)
y=y[:,-(future_pred):]
gt=torch.cat((data["x"][0].float(),data["y"][0,0:future_pred].float()),dim=0).numpy()
annual=torch.cat((data["annual_x"][0].float(),data["annual_y"][0,0:future_pred].float()),dim=0).numpy()
pred=torch.cat((data["x"][0].float().to(device),y[0]),dim=0).cpu().detach().numpy()
plt.figure(figsize=(15,7))
plt.plot(pred,label='prediction',color='red')
plt.plot(annual,label='annual',color='green')
plt.plot(gt,label='gt',color='blue')
plt.legend()
plt.title("Error : {}".format(np.abs(pred[-1]-gt[-1])))
plt.show()